In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv, re
import nltk
import string
from html.parser import HTMLParser
import pickle
import logging

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.decomposition import TruncatedSVD, NMF
from gensim import corpora, models, similarities, matutils


/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [8]:
df = pd.read_pickle('tweet_data.pkl')

In [10]:
nltk.download('wordnet')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leesurkis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Count Vectorizer

In [11]:
cv = CountVectorizer(stop_words='english')

In [12]:
X = cv.fit_transform(df.stemmed)
count_vecs = pd.DataFrame(X.toarray(), index=df.stemmed, columns=cv.get_feature_names())

In [13]:
count_vecs.shape

(3613, 6927)

In [14]:
count_vecs.head()

,aaa,aaron,aateam,abbys,aber,aberdeen,abhijit,ability,abla,able,...,zero,zeth,zika,zilch,zionist,zombie,zombies,zone,zoolander,zumba
stemmed,,,,,,,,,,,,,,,,,,,,,
fuck heck move fridge knock landlord door angry mad,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
indian uber driver just call word wasnt move vehicle jump disgust,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ask parcel deliver pick store address fume poorcustomerservice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
whichever butt wipe pull alarm davis sound asleep piss angry upset tire sad tire hangry,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dont join phone talk rude take money acc willynilly fume,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### TF-IDF Vectorizer

In [15]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=10000)

X = word_vectorizer.fit_transform(df.stemmed)

# char_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='char',
#     ngram_range=(1, 6),
#     max_features=30000)
# char_vectorizer.fit(df.stemmed)

In [16]:
# wordvec = TfidfVectorizer(ngram_range=(1,2), binary=True, stop_words='english')

In [46]:
# word_vectorizer.get_feature_names()

In [18]:
tfidf_vecs = pd.DataFrame(X.toarray(), index=df.stemmed, columns=word_vectorizer.get_feature_names())

In [19]:
tfidf_vecs.head()

,aaa,aaron,aateam,abbys,aber,aberdeen,abhijit,ability,abla,able,...,zero,zeth,zika,zilch,zionist,zombie,zombies,zone,zoolander,zumba
stemmed,,,,,,,,,,,,,,,,,,,,,
fuck heck move fridge knock landlord door angry mad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
indian uber driver just call word wasnt move vehicle jump disgust,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ask parcel deliver pick store address fume poorcustomerservice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
whichever butt wipe pull alarm davis sound asleep piss angry upset tire sad tire hangry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dont join phone talk rude take money acc willynilly fume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### LSA on Count Vectorizer

In [20]:
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(count_vecs)
lsa.explained_variance_ratio_

array([0.00885203, 0.01269675, 0.0100346 , 0.00874681])

In [21]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2","component_3", "component_4"],
             columns = cv.get_feature_names())
topic_word

,aaa,aaron,aateam,abbys,aber,aberdeen,abhijit,ability,abla,able,...,zero,zeth,zika,zilch,zionist,zombie,zombies,zone,zoolander,zumba
component_1,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.003,0.002,0.002,...,0.005,0.0,0.0,0.0,0.0,0.001,0.000,0.0,0.001,0.0
component_2,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000,-0.0,-0.001,-0.000,0.001,...,-0.001,-0.0,-0.0,-0.0,-0.0,-0.000,0.001,-0.0,0.003,-0.0
component_3,0.0,0.0,0.0,-0.0,0.0,0.000,-0.0,-0.001,0.003,0.003,...,-0.005,0.0,-0.0,-0.0,0.0,-0.000,0.000,-0.0,-0.000,-0.0
component_4,0.0,0.0,-0.0,0.0,0.0,0.001,0.0,0.000,-0.001,-0.001,...,0.003,0.0,0.0,0.0,-0.0,0.001,0.000,0.0,0.000,0.0


In [22]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [23]:
display_topics(lsa, cv.get_feature_names(), 20)


Topic  0
just, like, dont, make, want, know, think, love, watch, say, people, time, feel, good, lively, need, day, really, happy, look

Topic  1
lively, watch, amaze, broadcast, musically, musicallyjh, follow, youve, series, long, horror, wonderful, season, glee, advise, gotham, fallenskies, syfy, jericho, heroes

Topic  2
like, make, look, feel, say, today, guy, smile, bad, old, sound, better, bully, day, rabid, polish, horrible, work, big, ask

Topic  3
dont, know, think, make, let, love, smile, people, day, youre, happy, fear, good, change, thank, things, worry, sad, world, expect


### LSA on TF-IDF 

In [24]:
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(tfidf_vecs)
lsa.explained_variance_ratio_

array([0.01462442, 0.0027854 , 0.00374852, 0.00361182])

In [25]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2","component_3", "component_4"],
             columns = word_vectorizer.get_feature_names())
topic_word

,aaa,aaron,aateam,abbys,aber,aberdeen,abhijit,ability,abla,able,...,zero,zeth,zika,zilch,zionist,zombie,zombies,zone,zoolander,zumba
component_1,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000,0.000,...,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.002,0.000
component_2,0.0,0.001,0.001,0.001,0.0,0.002,0.0,0.006,0.002,0.003,...,0.005,0.0,0.0,0.001,0.0,0.003,0.0,0.001,0.001,0.000
component_3,-0.0,0.000,-0.003,-0.000,-0.0,0.001,-0.0,-0.003,-0.001,-0.002,...,0.002,-0.0,0.0,-0.001,-0.0,-0.001,-0.0,-0.001,-0.000,-0.001
component_4,-0.0,0.000,-0.001,0.000,-0.0,0.002,-0.0,0.010,0.002,0.002,...,-0.005,0.0,0.0,0.000,-0.0,-0.000,0.0,0.000,-0.002,0.001


In [26]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [27]:
display_topics(lsa, word_vectorizer.get_feature_names(), 20)


Topic  0
lively, broadcast, musically, amaze, watch, musicallyjh, just, like, dont, do, time, listen, lovely, get, horror, follow, hilarious, thank, wait, start

Topic  1
dont, just, like, make, think, love, know, want, feel, get, do, good, people, day, go, happy, let, say, smile, start

Topic  2
dont, know, think, let, love, worry, bitter, youre, wanna, leave, smile, sad, provoke, make, shy, patronus, want, purpose, better, fear

Topic  3
make, happy, day, smile, good, love, let, youre, optimism, thank, birthday, today, look, quotesoup, decision, morning, joyful, bless, bday, angry


### NMF on Count Vectorizer

In [28]:
nmf_model = NMF(4)
doc_topic = nmf_model.fit_transform(count_vecs)

In [29]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2","component_3", "component_4"],
             columns = cv.get_feature_names())
topic_word

,aaa,aaron,aateam,abbys,aber,aberdeen,abhijit,ability,abla,able,...,zero,zeth,zika,zilch,zionist,zombie,zombies,zone,zoolander,zumba
component_1,0.0,0.000,0.0,0.001,0.0,0.000,0.0,0.013,0.000,0.000,...,0.027,0.000,0.000,0.000,0.0,0.002,0.000,0.000,0.000,0.000
component_2,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.006,...,0.000,0.000,0.000,0.000,0.0,0.000,0.005,0.000,0.012,0.000
component_3,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.001,0.012,0.013,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000
component_4,0.0,0.001,0.0,0.002,0.0,0.004,0.0,0.009,0.002,0.002,...,0.018,0.001,0.001,0.002,0.0,0.005,0.000,0.003,0.000,0.001


In [30]:
display_topics(nmf_model, cv.get_feature_names(), 15)


Topic  0
just, want, time, say, really, way, love, need, wish, didnt, start, tell, little, live, lol

Topic  1
lively, watch, amaze, broadcast, musically, time, musicallyjh, follow, wait, horror, youve, long, hilarious, series, season

Topic  2
like, feel, look, make, say, today, bad, guy, old, fuck, right, sound, work, bully, laugh

Topic  3
dont, make, know, think, love, let, people, day, smile, good, youre, happy, want, time, fear


### NMF on TF-IDF

In [31]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(tfidf_vecs)

In [32]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2","component_3", "component_4", "component_5"],
             columns = word_vectorizer.get_feature_names())
topic_word

,aaa,aaron,aateam,abbys,aber,aberdeen,abhijit,ability,abla,able,...,zero,zeth,zika,zilch,zionist,zombie,zombies,zone,zoolander,zumba
component_1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,...,0.000,0.000,0.0,0.000,0.0,0.000,0.001,0.000,0.006,0.000
component_2,0.001,0.001,0.004,0.002,0.001,0.000,0.001,0.006,0.000,0.002,...,0.018,0.001,0.0,0.003,0.0,0.006,0.000,0.002,0.003,0.000
component_3,0.000,0.001,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,...,0.008,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.001,0.000,0.001,0.000,0.002,0.000,0.021,0.004,0.004,...,0.000,0.001,0.0,0.001,0.0,0.003,0.001,0.001,0.000,0.002
component_5,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.005,0.006,...,0.000,0.000,0.0,0.000,0.0,0.001,0.000,0.000,0.000,0.000


In [33]:
display_topics(nmf_model, word_vectorizer.get_feature_names(), 15)


Topic  0
lively, broadcast, musically, amaze, watch, musicallyjh, listen, lovely, follow, horror, blake, pity, hilarious, buffnpolish, shower

Topic  1
just, get, go, want, time, say, do, people, need, really, start, lose, fuck, didnt, today

Topic  2
dont, know, think, let, worry, leave, bitter, sad, want, wanna, better, youre, tell, anger, shy

Topic  3
make, love, happy, day, smile, good, youre, thank, let, start, optimism, today, birthday, joyful, morning

Topic  4
like, feel, look, do, offend, depression, right, bad, worthless, optimism, today, literally, guy, laugh, old


### LDA on Count Vectorized

In [34]:
# create the term-doc matrix, transpose so that terms are rows

doc_word = cv.transform(count_vecs).transpose()

In [35]:
pd.DataFrame(doc_word.toarray(), cv.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,6917,6918,6919,6920,6921,6922,6923,6924,6925,6926
aaa,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaron,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aateam,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbys,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aber,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
doc_word.shape

(6927, 6927)

In [37]:
# converts sparse matrix of counts to a gensim corpus 
corpus = matutils.Sparse2Corpus(doc_word)

In [38]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [39]:
len(id2word)

6927

In [40]:
# 'fitting' the model
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=5)

2019-05-21 12:49:35,019 : INFO : using symmetric alpha at 0.3333333333333333
2019-05-21 12:49:35,020 : INFO : using symmetric eta at 0.3333333333333333
2019-05-21 12:49:35,022 : INFO : using serial LDA version on this node
2019-05-21 12:49:35,027 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 6927 documents, updating model once every 2000 documents, evaluating perplexity every 6927 documents, iterating 50x with a convergence threshold of 0.001000
2019-05-21 12:49:35,032 : INFO : PROGRESS: pass 0, at document #2000/6927
2019-05-21 12:49:35,735 : INFO : merging changes from 2000 documents into a model of 6927 documents
2019-05-21 12:49:35,738 : INFO : topic #0 (0.333): 0.001*"bulushi" + 0.001*"dfs" + 0.001*"customerservice" + 0.001*"bbcfootball" + 0.001*"admit" + 0.001*"express" + 0.001*"course" + 0.001*"apprehension" + 0.001*"binary" + 0.001*"bojack"
2019-05-21 12:49:35,739 : INFO : topic #1 (0.333): 0.001*"airplane" + 0.001*"aww" + 0.0

In [41]:
lda.print_topics()

2019-05-21 12:49:43,144 : INFO : topic #0 (0.333): 0.001*"weak" + 0.001*"wespernalsontoo" + 0.001*"vast" + 0.001*"update" + 0.001*"treatcustomersfairly" + 0.001*"unacceptable" + 0.001*"younger" + 0.001*"unprepared" + 0.001*"violationnn" + 0.001*"youve"
2019-05-21 12:49:43,145 : INFO : topic #1 (0.333): 0.000*"photos" + 0.000*"phew" + 0.000*"saudi" + 0.000*"ruffle" + 0.000*"relate" + 0.000*"qualm" + 0.000*"sydney" + 0.000*"potential" + 0.000*"sleepingn" + 0.000*"paltry"
2019-05-21 12:49:43,146 : INFO : topic #2 (0.333): 0.000*"shawty" + 0.000*"strawberry" + 0.000*"ordunno" + 0.000*"patroons" + 0.000*"signal" + 0.000*"remark" + 0.000*"superexcited" + 0.000*"norffampton" + 0.000*"patheticservices" + 0.000*"rudely"


[(0,
  '0.001*"weak" + 0.001*"wespernalsontoo" + 0.001*"vast" + 0.001*"update" + 0.001*"treatcustomersfairly" + 0.001*"unacceptable" + 0.001*"younger" + 0.001*"unprepared" + 0.001*"violationnn" + 0.001*"youve"'),
 (1,
  '0.000*"photos" + 0.000*"phew" + 0.000*"saudi" + 0.000*"ruffle" + 0.000*"relate" + 0.000*"qualm" + 0.000*"sydney" + 0.000*"potential" + 0.000*"sleepingn" + 0.000*"paltry"'),
 (2,
  '0.000*"shawty" + 0.000*"strawberry" + 0.000*"ordunno" + 0.000*"patroons" + 0.000*"signal" + 0.000*"remark" + 0.000*"superexcited" + 0.000*"norffampton" + 0.000*"patheticservices" + 0.000*"rudely"')]

In [42]:
lda_corpus = lda[corpus]
lda_corpus

In [43]:
lda_docs = [doc for doc in lda_corpus]

In [44]:
lda_docs[0:5]

[[(0, 0.6505064), (1, 0.17545708), (2, 0.17403653)],
 [(0, 0.16845603), (1, 0.6616514), (2, 0.16989258)],
 [(0, 0.1687343), (1, 0.17102896), (2, 0.6602368)],
 [(0, 0.6505465), (1, 0.17542997), (2, 0.17402358)],
 [(0, 0.16873457), (1, 0.17103206), (2, 0.6602334)]]

In [45]:
count_vecs[1]

KeyError: 1